In [2]:
import pypsa
import pandas as pd
import numpy as np

c:\Users\long.nguyen\anaconda3\envs\pypsa-agora\lib\site-packages\pypsa\networkclustering.py:16: UserWarning: The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 
  warnings.warn(


In [5]:
solver = "cbc"

*** 
Check-point 1:

**Build a network in PyPSA with one node (`bus`) named "Utopia" and attach an electricity load (`load`) to it. For simplicity, we assume the load profile to be flat for now. Afterwards, we want to supply electricy by attaching one gas-fired power plant implemented as (`generator`) (you have to call [`network.set_snapshots`](https://pypsa-readthedocs.readthedocs.io/en/readthedocs/api_reference.html?highlight=set_snapshots) to select a year). As help you should have a look at the [PyPSA documentation](https://pypsa-readthedocs.readthedocs.io/en/readthedocs/index.html) and the [minimal lopf example](https://www.pypsa.org/examples/minimal_example_lopf.html), understand what the [components documentation](https://pypsa-readthedocs.readthedocs.io/en/readthedocs/components.html) of PyPSA gives you and that you can find the underlying objective function and constraints in the [LOPF documentation](https://pypsa-readthedocs.readthedocs.io/en/readthedocs/optimal_power_flow.html).**

> **Remarks:** For time reasons, you do not have to build the network from scratch. However, to get you acquainted with PyPSA we have omitted a few elements or some of the parameters of the network marked by three question marks `???`. Either, you have to add an element similar to the one in the box above or add a few parameters.

#### Initialize network

In [2]:
# Create empty PyPSA network
network = pypsa.Network()

In [ ]:
# Set snapshots to the year 2023 and at hourly resolution
snapshots = pd.date_range(???, ???, freq=???, inclusive=???)
network.set_snapshots(snapshots)

Add `Utopia` bus with electricity as `carrier`

In [ ]:
???

Add constant hourly electricity load of `100MW`

In [ ]:
???

Add a gas generator at the electricity bus with a capacity of 200MW and marginal cost of 20$/MWh to the network?

In [ ]:
# Creating gas-fired powerplant
network.add(
    class_name="Generator",
    name="gas_power_plant",
    bus=???,
    carrier="gas",
    ???=200,
    ???=20,  # marginal cost of gas-fired power plant ($/MWh)
)


Now try to solve your network

In [ ]:
# Solve network using cbc solver
network.optimize(pyomo=False, solver_name="cbc")

Lets look at some results! Back to read the docs. How would you look at results?

In [ ]:
???

Ending of check-point 1 - export network

In [ ]:
# Export network
network.export_to_netcdf("../results/network_c1.nc")

***
Check-point 2:

**Remove the generator from the previous exercise and replace it with two actual power plants: one nuclear and one gas-fired. The annualized capital costs and marginal costs for these plants should be calculated using the provided input parameters. Instead of setting the plant capacities exogenously, allow the model to determine the optimal capacities based on the LOPF.**

Beginning

In [ ]:
# Import check-point 1 network
network = pypsa.Network("../results/network_c1.nc")

In [ ]:
# remove generator from previous check-point
network.remove(class_name="Generator", name="gas_power_plant")

Add a nuclear powerplant to your network based on provided CAPEX, FOM, VOM, fuel_cost, efficiency, interest rate and lifetime.

> **Source:** all costs for the example are taken from PyPSA's technology database (https://github.com/PyPSA/technology-data)

In [9]:
# Nuclear power plant techno-economic parameters are given as:
lifetime = 40
interest = 0.05
CAPEX = 9453549 # $/MW
FOM = 120060  # $/MW fixed
VOM = 3.9 # $/MWh variable
fuel_cost = 3.75 # $/MWhth per unit Uranium consumed
efficiency = 0.326

In [ ]:
# We need to calculate annualized capital expenditure
def calculate_annualised_capex(capex: float, interest: float, lifetime: int):
    crf = (
        interest * (1 + interest) ** lifetime / ((1 + interest) ** lifetime - 1)
    )  # Capital recovery factor
    return capex * crf

In [ ]:
annualized_capex = calculate_annualised_capex(CAPEX, interest, lifetime)

network.add(
    class_name="Generator",
    name="nuclear_power_plant",
    bus=???,
    marginal_cost=???, #$/MWh
    capital_cost=???, #$/MW
    p_nom_extendable=???,  # Allow endogenous investment
    efficiency=efficiency, 
)

Add a gas-fired power plant to your network

In [ ]:
# Gas-fired power plant techno-economic parameters are given as:
lifetime = 25
interest = 0.05
CAPEX = 995257 # $/MW
FOM = 33234 # $/MW fixed
VOM = 5  # $/MWh variable
fuel_cost = 24.57 # $/MWhth - per unit gas consumed
efficiency = 0.57

In [ ]:
???

Ending of check-point 2 - Solve network, analyse results and export network

In [ ]:
# Solve network using cbc solver
network.optimize(pyomo=False, solver_name="cbc")

Look at your results! 

In [ ]:
???

In [ ]:
# Export network
network.export_to_netcdf("../results/network_c2.nc")

***
Check-point 3:

**Add a VRE power plant to the network: Solar PV with extendable capacity.** 

**Hint:** Solar PV can only provide electricity during day time and its generation pattern depends on solar irradiation**

In [ ]:
# Import check-point 2 network
network = pypsa.Network("../results/network_c2.nc")

Import an example solar daily availability and assume same pattern throughout the year

In [4]:
# Import an example daily pattern
solar_pattern = pd.read_csv("../data/weather data/example_solar_daily_pattern.csv")["daily pattern"]
# annual time-series availability of solar farm(just a simplified example)
yearly_avail = pd.Series(list(solar_pattern) *365, index=network.snapshots)

Add a solar farm to your network

In [ ]:
# Solar farm techno-economic parameters are given as:
lifetime = 37.5
interest = 0.05
CAPEX = 744227  # $/MW
FOM = 12856.5  # $/MW fixed
VOM = 0.0106  # $/MWh variable
fuel_cost = 0  # per unit of sun consumed

In [ ]:
???

network.add(
    class_name="Generator",
    name="Solar_farm",
    bus=???,
    marginal_cost=???,
    capital_cost=???,
    ???=True,
    ???=yearly_avail,  # Solar farm availability
)

Ending of check-point 3 - Solve network, analyse results and export network

In [ ]:
# Solve network using cbc solver
network.optimize(pyomo=False, solver_name="cbc")

In [ ]:
# Inspect solar farm interaction with other powerplants to supply loads
???

In [ ]:
# Exporting check-point 3 network
network.export_to_netcdf("../results/network_c3.nc")